In [43]:
from collections import deque
from abc import ABC, abstractmethod

class InvalidExpressionException(Exception):
    def __init__(self, message, error_code):
        super().__init__(message)
        self.error_code = error_code

class Node(ABC):
    @abstractmethod
    def evaluate(self) -> int:
        pass

    @staticmethod
    def from_string(data: str) -> 'Node':
        match data:
            case "*":
                return MulNode()
            case "/":
                return DivNode()
            case "+":
                return AddNode()
            case "-":
                return SubNode()
            case _:
                return NumericNode(data)

## Abstract operator node
class OperatorNode(Node):
    def __init__(self):
        self.left: Node = None
        self.right: Node = None

class NumericNode(Node):
    def __init__(self, num: str):
        self.number = int(num)
    
    def evaluate(self) -> int:
        return self.number

class MulNode(OperatorNode):
    def evaluate(self) -> int:
        return self.left.evaluate() * self.right.evaluate()

class DivNode(OperatorNode):
    def evaluate(self) -> int:
        return self.left.evaluate() / self.right.evaluate()

class AddNode(OperatorNode):
    def evaluate(self) -> int:
        return self.left.evaluate() + self.right.evaluate()

class SubNode(OperatorNode):
    def evaluate(self) -> int:
        return self.left.evaluate() - self.right.evaluate()


class Calculator:
    def __init__(self, parser: PostfixParser):
        self.parser = parser
        
    def calculate(self, infix: str):
        postfix = self.parser.parse(infix)
        stack = deque()
        digit = ""
        for ch in postfix:
            if ch.isdigit():
                digit += ch
            elif ch == ';':
                stack.append(Node.from_string(digit))
                digit = ""
            else:
                node = Node.from_string(str(ch))
                if isinstance(node, OperatorNode):
                    node.right = stack.pop()
                    node.left = stack.pop()
                    stack.append(node)
        return stack.pop().evaluate()

class PostfixParser:
    def isValid(self, ch: chr) -> bool:
        return ch == '(' or ch == ')' or ch == '^' or ch == '*' or ch =='/' or ch == '+' or ch == '-' or (ch >= '0' and ch <= '9')
        
    def precendence(self, operator: chr) -> int:
        if operator == '^':
            return 3
        elif operator == '*' or operator == '/':
            return 2
        elif operator == '+' or operator == '-':
            return 1
        else:
            return -1
    
    def parse(self, infix: str) -> str:
        buffer = ""
        stack = deque()
        digit = ""
        for ch in infix:
            if not self.isValid(ch):
                raise InvalidExpressionException(f"Sorry, invalid char {ch}", 400)
            elif ch.isdigit():
                 digit += ch
            else:
                if digit:
                    buffer += (digit + ";")
                digit = ""

                if ch == '(':
                    stack.append(ch)
                elif ch == ')':
                    while stack and stack[-1] != '(':
                        buffer += stack.pop()
                    stack.pop() ## remove '(' from expression
                else:
                    while stack and self.precendence(ch) <= self.precendence(stack[-1]):
                        buffer += stack.pop()
                    stack.append(ch)
        if digit:
            buffer += (digit + ";")
        while stack:
            buffer += stack.pop()
            
        return buffer      


if __name__ == '__main__':
    while True:
        expression=input("Please enter expression or `e` for exit: ")
        match expression:
            case "e":
                print("Exiting!!")
                break;
            case _:
                try:
                    parser = PostfixParser()
                    calculator = Calculator(parser)
                    print(calculator.calculate(expression))
                except Exception as e:
                    print(f"Exception : \"{e}\"")
                    raise e

Please enter expression or `e` for exit:  1+(3-2)*9/2


5.5


Please enter expression or `e` for exit:  2+2


4


Please enter expression or `e` for exit:  (1+(4+5+2)-3)+(6+8)


23


Please enter expression or `e` for exit:  24/12+1


3.0


Please enter expression or `e` for exit:  e


Exiting!!
